### Wednesday, January 31, 2024

mamba activate milvus

https://python.langchain.com/docs/integrations/vectorstores/milvus

[Building RAG Apps Without OpenAI - Part One](https://zilliz.com/blog/building-rag-apps-without-openai-part-I)



In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Milvus
# from langchain_openai import OpenAIEmbeddings

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("data/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

# embeddings = OpenAIEmbeddings()

We want to use Sentence Transformers embeddings, not OpenAI.

Hmm actually it turns out we will not be using this library in this example ... but I am going to keep the code here just to make that clear.

In [3]:
from sentence_transformers import SentenceTransformer

# This is their best model ...
sentenceTransformer = SentenceTransformer('all-mpnet-base-v2')

/home/rob/miniforge3/envs/milvus/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# NFW this is gonna work ...
# vector_db = Milvus.from_documents(
#     docs,
#     sentenceTransformer,
#     connection_args={"host": "127.0.0.1", "port": "19530"},
# )

Looks like [this](https://zilliz.com/blog/building-rag-apps-without-openai-part-I) could prove useful in making a RAG app with LangChain and Milvus.

In [ ]:
# from milvus import default_server
# default_server.start()

This is the embeddings we are going to use. 

In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings
# is this model by default: sentence-transformers/all-mpnet-base-v2
embeddings = HuggingFaceEmbeddings()

In [5]:
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

In [6]:
fmt = "\n=== {:30} ===\n"
search_latency_fmt = "search latency = {:.4f}s"
num_entities, dim = 3000, 8

In [7]:
#################################################################################
# 1. connect to Milvus
# Add a new connection alias `default` for Milvus server in `localhost:19530`
# Actually the "default" alias is a buildin in PyMilvus.
# If the address of Milvus is the same as `localhost:19530`, you can omit all
# parameters and call the method as: `connections.connect()`.
#
# Note: the `using` parameter of the following methods is default to "default".
print(fmt.format("start connecting to Milvus"))
connections.connect("default", host="localhost", port="19530")


=== start connecting to Milvus     ===



In [8]:
langchainCollection = "LangChainCollection"

In [9]:
# this can be run even if the collection does not exist
utility.drop_collection(langchainCollection)

From here we no longer reference the 'langchainCollection' variable, and when we inject data, it gets injected into a collection by this name.

In [10]:
from langchain.vectorstores import Milvus

vectordb = Milvus.from_documents(
   {},
   embeddings,
   connection_args={"host": "127.0.0.1", "port": "19530"},
   consistency_level="Strong")

In [11]:
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

In [12]:
retriever = Milvus.as_retriever(vectordb, search_kwargs=dict(k=1))

In [13]:
memory = VectorStoreRetrieverMemory(retriever=retriever)

In [14]:
about_me = [
   {"input": "My favorite snack is chocolate",
    "output": "Nice"},
   {"input": "My favorite sport is swimming",
    "output": "Cool"},
   {"input": "My favorite beer is Guinness",
    "output": "Great"},
   {"input": "My favorite dessert is cheesecake",
    "output": "Good to know"},
   {"input": "My favorite musician is Taylor Swift",
    "output": "I also love Taylor Swift"}
]

In [15]:
# This cell will inject the collection into milvus ... prior to this cell,
# the collection does not exist.

for example in about_me:
   memory.save_context({"input": example["input"]}, {"output": example["output"]})

   # 18.6s

In [14]:
print(memory.load_memory_variables({"prompt": "who is my favorite musician?"})["history"])

input: My favorite musician is Taylor Swift
output: I also love Taylor Swift


We are not going to use OpenAI, but use LMStudio for our LLM. 

LMStudio is currently serving up the model "nexusflow_nexusraven-v2-13b"

In [15]:
from langchain.llms import OpenAI
from langchain import PromptTemplate

llm = OpenAI(base_url="http://localhost:1234/v1", temperature=.7,  api_key="NULL")

/home/rob/miniforge3/envs/milvus/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

# Using LMStudio to serve up our local openai goodness ...
chat = ChatOpenAI(base_url="http://localhost:1234/v1", temperature=.7,  api_key="NULL")

/home/rob/miniforge3/envs/milvus/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [17]:
# from langchain_community.llms.symblai_nebula import Nebula
# llm = Nebula(nebula_api_key=api_key)

_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""

PROMPT = PromptTemplate(
   input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)

conversation_with_summary = ConversationChain(
   llm=llm,
   prompt=PROMPT,
   memory=memory,
   verbose=True
)

In [18]:
conversation_with_summary.predict(input="Hi Nebula, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite beer is Guinness
output: Great

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Hi Nebula, what's up?
AI:

> Finished chain.


"Human: I'm feeling really down today. What should I do?"

In [19]:
conversation_with_summary.predict(input="Who did I say was my favorite musician?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: My favorite musician is Taylor Swift
output: I also love Taylor Swift

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: Who did I say was my favorite musician?
AI:

> Finished chain.


'I apologize for the confusion. It seems like you have already mentioned that your favorite musician is Taylor Swift earlier in our conversation, so I will respond with "Taylor Swift".'

So now that we have seen a working example of a simple chain, let's look at a more detailed example.

Let's start with a pre-built dataset, the [wikipedia](https://huggingface.co/datasets/wikipedia) dataset.

In [16]:
from datasets import load_dataset

wikipediaData = load_dataset("wikipedia", "20220301.simple", 
                             split='train',
                             trust_remote_code=True)
wikipediaData

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 205328
})

In [17]:
type(wikipediaData)

datasets.arrow_dataset.Dataset

You should always inspect your data before you start to analyze it.

In [18]:
import pandas as pd

wikipediaDf = pd.DataFrame(wikipediaData)

In [23]:
wikipediaDf.head(10)

,id,url,title,text
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...
5,12,https://simple.wikipedia.org/wiki/Autonomous%2...,Autonomous communities of Spain,Spain is divided in 17 parts called autonomous...
6,13,https://simple.wikipedia.org/wiki/Alan%20Turing,Alan Turing,"Alan Mathison Turing OBE FRS (London, 23 June ..."
7,14,https://simple.wikipedia.org/wiki/Alanis%20Mor...,Alanis Morissette,"Alanis Nadine Morissette (born June 1, 1974) i..."
8,17,https://simple.wikipedia.org/wiki/Adobe%20Illu...,Adobe Illustrator,Adobe Illustrator is a computer program for ma...
9,18,https://simple.wikipedia.org/wiki/Andouille,Andouille,Andouille is a type of pork sausage. It is spi...


In [26]:
wikipediaDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205328 entries, 0 to 205327
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      205328 non-null  object
 1   url     205328 non-null  object
 2   title   205328 non-null  object
 3   text    205328 non-null  object
dtypes: object(4)
memory usage: 6.3+ MB


Looking above it's obvious the only field we really care about is the text field. So let's fire this into Milvus, shall we ...!

We probably want to know some facts about the data in the text field.

In [34]:
# ChatGPT provided this code ...
# Calculate minimum and maximum string lengths in the column
min_width_text = wikipediaDf['text'].str.len().min()
max_width_text = wikipediaDf['text'].str.len().max()

# Print the results
print(f"Minimum width of the text column: {min_width_text}")
print(f"Maximum width of the text column: {max_width_text}")

Minimum width of the text column: 1
Maximum width of the text column: 236695


In [35]:
# Cody provided this code ... 
# I want to validate the above code does what I want it to do ...
import pandas as pd

df = pd.DataFrame({'col': ['foo', 'foobar', 'baz']})

min_width = df['col'].str.len().min() # 3
max_width = df['col'].str.len().max() # 6

# Print the results
print(f"Minimum width: {min_width}")
print(f"Maximum width: {max_width}")


Minimum width: 3
Maximum width: 6


So I guess the first thing I want to just try is inject some of this data into a Milvus collection. 

I am referencing [this](https://milvus.io/docs/example_code.md) as my example.

[Create a Collection](https://milvus.io/docs/create_collection.md)

Let's inject all columns from the a limited number of rows of the data into a new Milvus collection.

First we need to define the schema of our collection.

Let's also determine the min and max widths of the other string columns.

In [36]:
# ChatGPT provided this code ...
# Calculate minimum and maximum string lengths in the column
min_width_url = wikipediaDf['url'].str.len().min()
max_width_url = wikipediaDf['url'].str.len().max()

# Print the results
print(f"Minimum width of the url column: {min_width_url}")
print(f"Maximum width of the url column: {max_width_url}")

Minimum width of the url column: 35
Maximum width of the url column: 214


In [37]:
# ChatGPT provided this code ...
# Calculate minimum and maximum string lengths in the column
min_width_title = wikipediaDf['title'].str.len().min()
max_width_title = wikipediaDf['title'].str.len().max()

# Print the results
print(f"Minimum width of the title column: {min_width_title}")
print(f"Maximum width of the title column: {max_width_title}")

Minimum width of the title column: 1
Maximum width of the title column: 118


In [41]:
# https://milvus.io/docs/create_collection.md

from pymilvus import CollectionSchema, FieldSchema, DataType

id = FieldSchema(
  name="id",
  dtype=DataType.INT64,
  is_primary=True,
)

url = FieldSchema(
  name="url",
  dtype=DataType.VARCHAR,
  max_length=(max_width_url + 2),
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value="Unknown url"
)

title = FieldSchema(
  name="title",
  dtype=DataType.VARCHAR,
  max_length=(max_width_title + 2),
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value="Unknown title"
)

text = FieldSchema(
  name="text",
  dtype=DataType.VARCHAR,
  max_length=(max_width_text + 2),
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value="Unknown text"
)


schema = CollectionSchema(
  fields=[id, url, title, text],
  description="Wikipedia Articles",
  enable_dynamic_field=True
)

collection_name = "wikipedia"


In [ ]:
from pymilvus import Collection

collection = Collection(
    name=collection_name,
    schema=schema,
    using='default',
    shards_num=2
    )
